In [2]:
from nltk.tokenize import TreebankWordTokenizer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from collections import defaultdict
from collections import Counter
import math
import time
import csv
import re
import random

In [3]:
# ['favorited', 'retweetCount', 'created', 'text', 'isRetweet', 'truncated', 'retweeted', 'replyToUID', 
# 'longitude', 'replyToSN', 'label', 'replyToSID', 'latitude', 'statusSource', 'favoriteCount', 'id', 'screenName']
with open('all/train.csv', 'rb') as csvfile:
    tweets = csv.DictReader(csvfile)
    tweets = list(tweets)
    l = len(tweets)

# v_indices = random.sample(range(l), 100)
# t_indices = [i for i in range(l) if i not in v_indices]


android = [(t['text'], 'android') for t in tweets if 'Android' in t['statusSource']]
iphone = [(t['text'], 'iphone') for t in tweets if 'iPhone' in t['statusSource']]


# validate_content = [t['text'] for t in tweets[c_index:]]
# validate_content2 = [(t['text'], t['statusSource']) for t in tweets[c_index:]]

# for t in tweets[0:c_index]:
#     train_data = [t['text'] for t in tweets[0:c_index]]
#     train_data2 = [(t['text'], t['statusSource']) for t in tweets[0:c_index]]

In [4]:
tweets = []
for (words, sentiment) in android + iphone:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 3]
    tweets.append((words_filtered, sentiment))

In [5]:
#extracting features now
def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

word_features = get_word_features(get_words_in_tweets(tweets))

In [6]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [7]:
training_set = nltk.classify.apply_features(extract_features, tweets)
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [13]:
with open('all/test.csv', 'rb') as csvfile:
    test = csv.DictReader(csvfile)
    test = list(test)
    test_tweets = [t['text'] for t in test]

In [16]:
#1 for android, -1 for iphone
results = []
for i in range(len(test_tweets)):
    sentiment = classifier.classify(extract_features(test_tweets[i].split()))
    if '#' in test_tweets[i]:
        results.append((i, 'iPhone'))
    elif sentiment == 'android' or '"' in test_tweets[i]: 
        results.append((i,'Android'))
    else: 
        results.append((i, 'iPhone'))

In [26]:
with open('final_submission.csv', mode='w') as final:
    final_writer = csv.writer(final, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    final_writer.writerow(['ID', 'Label'])
    for r in results:
        if r[1] == 'Android':
            final_writer.writerow([r[0], 1])
        elif r[1] == 'iPhone':
            final_writer.writerow([r[0], -1])

In [24]:
results

[(0, 'iPhone'),
 (1, 'iPhone'),
 (2, 'iPhone'),
 (3, 'Android'),
 (4, 'Android'),
 (5, 'Android'),
 (6, 'iPhone'),
 (7, 'iPhone'),
 (8, 'Android'),
 (9, 'iPhone'),
 (10, 'Android'),
 (11, 'Android'),
 (12, 'Android'),
 (13, 'Android'),
 (14, 'iPhone'),
 (15, 'Android'),
 (16, 'iPhone'),
 (17, 'iPhone'),
 (18, 'iPhone'),
 (19, 'Android'),
 (20, 'Android'),
 (21, 'Android'),
 (22, 'Android'),
 (23, 'Android'),
 (24, 'iPhone'),
 (25, 'iPhone'),
 (26, 'iPhone'),
 (27, 'iPhone'),
 (28, 'iPhone'),
 (29, 'iPhone'),
 (30, 'iPhone'),
 (31, 'iPhone'),
 (32, 'iPhone'),
 (33, 'Android'),
 (34, 'iPhone'),
 (35, 'iPhone'),
 (36, 'iPhone'),
 (37, 'iPhone'),
 (38, 'Android'),
 (39, 'iPhone'),
 (40, 'iPhone'),
 (41, 'iPhone'),
 (42, 'iPhone'),
 (43, 'iPhone'),
 (44, 'iPhone'),
 (45, 'iPhone'),
 (46, 'iPhone'),
 (47, 'Android'),
 (48, 'iPhone'),
 (49, 'iPhone'),
 (50, 'iPhone'),
 (51, 'iPhone'),
 (52, 'Android'),
 (53, 'iPhone'),
 (54, 'iPhone'),
 (55, 'iPhone'),
 (56, 'iPhone'),
 (57, 'Android'),
 (58,